# DONE:

- x include non-Metaphor data and label

- repeat experiments after sampling original sentences, not random sampling

- fix the damn indentation on the accuracy

- Try freezing some layers. Hopefully reduces training time (significantly)

- FIXING BERT TOKENIZER TO REDEWIEDERGABE!!!

- make use of cuda or mps nicer

- increase freeze rate (Dropped performance slightly)

- add additional dropout layer (not really sure if it helped)

# NOW:

- include loss and score plots

# TODO:
    
- include F1-score in evaluation

- x test F1-score loss function
    - https://www.kaggle.com/code/bigironsphere/loss-function-library-keras-pytorch/notebook
    - not avaiable for mps yet!


In [16]:
import numpy as np
import pandas as pd
import torch

import matplotlib.pyplot as plt
import seaborn as sns

import os
import re

np.random.seed(3093453)
sns.set(context="talk", style="darkgrid")

In [29]:
cwd = os.getcwd()
folder = "unclean+translated-data"
file = "unclean-de.csv"
path = "/".join( (cwd, folder, file) )

df = pd.read_csv(path, index_col=0)
df["lang"] = "de"

files = os.listdir(folder)
files = [file for file in files if re.match(r'unclean-[a-z]{2}-de.csv', file)]

for file in files:
    path = "/".join( (cwd, folder, file) )
    data = pd.read_csv(path, index_col=0)
    data["lang"] = file[8:10]
    
    df = pd.concat([df, data])

del(data)
df.reset_index(inplace=True)
df

index                                         Textstelle  Metapher  \
0         0   "Der politische Körper verwendet in beiden Fä...         3   
1         1   "Immer noch wird durch die protestantische Le...         0   
2         2   "Kein Zweifel", schreibt Alb. Schaeffle 1), v...         0   
3         3   "Mit je weiterem Blick wir die Stoffwelt zu ü...         1   
4         4   "Solange das Menschengeschlecht seinen Zerstö...         0   
...     ...                                                ...       ...   
1593    794  Geschichte Sie wollte Naturforschung im beschr...         0   
1594    795  wenn wir an ein Prinzip denken und darauf aufb...         1   
1595    796  Sieht man von dem Schimpfwort „Rechtsphilosoph...         3   
1596    797  „Die Macht, die das Kabinett hat, um die versc...         0   
1597    798  Über die Auswirkungen der Kreuzung auf die kör...         0   

      Kandidat  Nein          Metapher?  \
0            1     0           Metapher   
1            4     0  Metaphernkandidat   
2            4     0  Metaphernkandidat   
3            3     0  Metaphernkandidat   
4            4     0  Metaphernkandidat   
...        ...   ...                ...   
1593         4     0  Metaphernkandidat   
1594         3     0  Metaphernkandidat   
1595         1     0           Metapher   
1596         2     0                NaN   
1597         2     0                NaN   

                                                  Fokus  \
0     ['Zähnen und Krallen', 'Zähnen und Krallen', n...   
1                                  [nan, nan, nan, nan]   
2                                  [nan, nan, nan, nan]   
3     [nan, 'Stoffwelt, verzwergt und entkleidet', n...   
4                                  [nan, nan, nan, nan]   
...                                                 ...   
1593                               [nan, nan, nan, nan]   
1594             [nan, nan, 'verstellbare Puppen', nan]   
1595  ['schwindsüchtig ist', nan, 'schwindsüchtig', ...   
1596                                         [nan, nan]   
1597                                         [nan, nan]   

                                                 Rahmen  \
0     ['der politische Körper', 'der politische Körp...   
1                                  [nan, nan, nan, nan]   
2                                  [nan, nan, nan, nan]   
3            [nan, 'unser praktisches Ideal', nan, nan]   
4                                  [nan, nan, nan, nan]   
...                                                 ...   
1593                               [nan, nan, nan, nan]   
1594  [nan, nan, 'staatliche Menschenverhältnisse', ...   
1595  ['daß eine Philosophie', nan, 'eine Philosophi...   
1596                                         [nan, nan]   
1597                                         [nan, nan]   

                            Annotator  \
0     ['B', 'K', 'A', 'GoldStandard']   
1     ['B', 'K', 'A', 'GoldStandard']   
2     ['B', 'K', 'A', 'GoldStandard']   
3     ['B', 'K', 'A', 'GoldStandard']   
4     ['B', 'K', 'A', 'GoldStandard']   
...                               ...   
1593  ['T', 'P', 'B', 'GoldStandard']   
1594  ['P', 'K', 'B', 'GoldStandard']   
1595  ['P', 'K', 'B', 'GoldStandard']   
1596                       ['P', 'B']   
1597                       ['B', 'P']   

                                                   orig lang  
0      "Der politische Körper verwendet in beiden Fä...   de  
1      "Immer noch wird durch die protestantische Le...   de  
2      "Kein Zweifel", schreibt Alb. Schaeffle 1), v...   de  
3      "Mit je weiterem Blick wir die Stoffwelt zu ü...   de  
4      "Solange das Menschengeschlecht seinen Zerstö...   de  
...                                                 ...  ...  
1593  verlaufs; sie hat Naturforschung in dem von un...   en  
1594  wenn wir uns ein Prinzip denken und auf Grund ...   en  
1595  wenn wir von dem schlechten Worte »Rechtsphilo...   en  
1596  »Die Macht, die das Kabinett 

In [30]:
folder = "clean+translated-data"
file = "non-metaphor-clean-de.csv"
path = "/".join( (cwd, folder, file) )

non = pd.read_csv(path)
non["lang"] = "de"
non

index                                         Textstelle  Metapher  \
0         0  Eine sammlung von preisscliristenz Herausgegeb...         0   
1         1  Januar  veröffentlichten die Professoren Haeck...         0   
2         2  Unter den  Abhandlungen und Schriften, welche ...         0   
3         3  In dem vorliegenden Sammelwerke werden sieben ...         0   
4         4  Es folgt hier zunächst das Preisausschreiben, ...         0   
...     ...                                                ...       ...   
3960   3960  In der Zeit des Humanismus und der Reformation...         0   
3961   3961  Den Mittelpunkt der Bildung stellten die alten...         0   
3962   3962  Jahrhunderts hat eine beständige Differenzieru...         0   
3963   3963  Die Pädagogik der Neuzeit scheint davon wenig ...         0   
3964   3964  Die Jugend in den Städten ist meistens unterer...         0   

      Kandidat  Nein Metapher?  Fokus  Rahmen Annotator  \
0            0     1      Nein    NaN     NaN         X   
1            0     1      Nein    NaN     NaN         X   
2            0     1      Nein    NaN     NaN         X   
3            0     1      Nein    NaN     NaN         X   
4            0     1      Nein    NaN     NaN         X   
...        ...   ...       ...    ...     ...       ...   
3960         0     1      Nein    NaN     NaN         X   
3961         0     1      Nein    NaN     NaN         X   
3962         0     1      Nein    NaN     NaN         X   
3963         0     1      Nein    NaN     NaN         X   
3964         0     1      Nein    NaN     NaN         X   

                                                   orig lang  
0     Eine sammlung von preisscliristenz Herausgegeb...   de  
1     Januar 1900 veröffentlichten die Professoren H...   de  
2     Unter den 60 Abhandlungen und Schriften, welch...   de  
3     In dem vorliegenden Sammelwerke werden sieben ...   de  
4     Es folgt hier zunächst das Preisausschreiben, ...   de  
...                                                 ...  ...  
3960  In der Zeit des Humanismus und der Reformation...   de  
3961  Den Mittelpunkt der Bildung stellten die alten...   de  
3962  Jahrhunderts hat eine beständige Differenzieru...   de  
3963  Die Pädagogik der Neuzeit scheint davon wenig ...   de  
3964  Die Jugend in den Städten ist meistens unterer...   de  

[3965 rows x 11 columns]

In [31]:
df = pd.concat([df, non])
df.reset_index(drop=True)
df.rename(columns= {"Kandidat": "Metaphernkandidat"}, inplace=True)
df

index                                         Textstelle  Metapher  \
0         0   "Der politische Körper verwendet in beiden Fä...         3   
1         1   "Immer noch wird durch die protestantische Le...         0   
2         2   "Kein Zweifel", schreibt Alb. Schaeffle 1), v...         0   
3         3   "Mit je weiterem Blick wir die Stoffwelt zu ü...         1   
4         4   "Solange das Menschengeschlecht seinen Zerstö...         0   
...     ...                                                ...       ...   
3960   3960  In der Zeit des Humanismus und der Reformation...         0   
3961   3961  Den Mittelpunkt der Bildung stellten die alten...         0   
3962   3962  Jahrhunderts hat eine beständige Differenzieru...         0   
3963   3963  Die Pädagogik der Neuzeit scheint davon wenig ...         0   
3964   3964  Die Jugend in den Städten ist meistens unterer...         0   

      Metaphernkandidat  Nein          Metapher?  \
0                     1     0           Metapher   
1                     4     0  Metaphernkandidat   
2                     4     0  Metaphernkandidat   
3                     3     0  Metaphernkandidat   
4                     4     0  Metaphernkandidat   
...                 ...   ...                ...   
3960                  0     1               Nein   
3961                  0     1               Nein   
3962                  0     1               Nein   
3963                  0     1               Nein   
3964                  0     1               Nein   

                                                  Fokus  \
0     ['Zähnen und Krallen', 'Zähnen und Krallen', n...   
1                                  [nan, nan, nan, nan]   
2                                  [nan, nan, nan, nan]   
3     [nan, 'Stoffwelt, verzwergt und entkleidet', n...   
4                                  [nan, nan, nan, nan]   
...                                                 ...   
3960                                                NaN   
3961                                                NaN   
3962                                                NaN   
3963                                                NaN   
3964                                                NaN   

                                                 Rahmen  \
0     ['der politische Körper', 'der politische Körp...   
1                                  [nan, nan, nan, nan]   
2                                  [nan, nan, nan, nan]   
3            [nan, 'unser praktisches Ideal', nan, nan]   
4                                  [nan, nan, nan, nan]   
...                                                 ...   
3960                                                NaN   
3961                                                NaN   
3962                                                NaN   
3963                                                NaN   
3964                                                NaN   

                            Annotator  \
0     ['B', 'K', 'A', 'GoldStandard']   
1     ['B', 'K', 'A', 'GoldStandard']   
2     ['B', 'K', 'A', 'GoldStandard']   
3     ['B', 'K', 'A', 'GoldStandard']   
4     ['B', 'K', 'A', 'GoldStandard']   
...                               ...   
3960                                X   
3961                                X   
3962                                X   
3963                                X   
3964                                X   

                                                   orig lang  
0      "Der politische Körper verwendet in beiden Fä...   de  
1      "Immer noch wird durch die protestantische Le...   de  
2      "Kein Zweifel", schreibt Alb. Schaeffle 1), v...   de  
3      "Mit je weiterem Blick wir die Stoffwelt zu ü...   de  
4      "Solange das Menschengeschlecht seinen Zerstö...   de  
...                                                 ...  ...  
3960  In der Zeit des Humanismus und der Reformation...   de  
3961  Den Mittelpunkt der Bildung stellten die alten.

## Sentence ID

We need to do this to make sure we dont have a german sentence in the train data
... and a english version in the test data, which skewes the results

(Essentially the model would just memorize seen points, rather than learn what a metaphor is)



In [32]:
labs = ["Metapher", "Metaphernkandidat", "Nein"]
labels = dict(zip(range(3), labs))

# impute missing labels with silver labels
missing_lab = df["Metapher?"].isnull()
df.loc[missing_lab, "Metapher?"] = [labels[i] for i in df.loc[missing_lab, labs].to_numpy().argmax(axis=1)]

In [33]:
folder = "clean+translated-data"
file = "total.csv"
path = "/".join( (cwd, folder, file) )

ID = pd.read_csv(path, index_col = 0)
ID = ID[["orig", "lang", "sentence_id"]]
ID

orig lang  sentence_id
0      "Der politische Körper verwendet in beiden Fä...   de            0
1      "Immer noch wird durch die protestantische Le...   de            1
2      "Kein Zweifel", schreibt Alb. Schaeffle 1), v...   de            2
3      "Mit je weiterem Blick wir die Stoffwelt zu ü...   de            3
4      "Solange das Menschengeschlecht seinen Zerstö...   de            4
...                                                 ...  ...          ...
3960  In der Zeit des Humanismus und der Reformation...   de         4759
3961  Den Mittelpunkt der Bildung stellten die alten...   de         4760
3962  Jahrhunderts hat eine beständige Differenzieru...   de         4761
3963  Die Pädagogik der Neuzeit scheint davon wenig ...   de         4762
3964  Die Jugend in den Städten ist meistens unterer...   de         4763

[7960 rows x 3 columns]

In [34]:
df = df.merge(ID, how = "left", on = ["lang", "orig"])

In [35]:
df[df.sentence_id.isnull()]

Empty DataFrame
Columns: [index, Textstelle, Metapher, Metaphernkandidat, Nein, Metapher?, Fokus, Rahmen, Annotator, orig, lang, sentence_id]
Index: []

In [36]:
folder = "unclean+translated-data"
file = "total.csv"
path = "/".join( (cwd, folder, file) )
df.to_csv(path)